In [ ]:
import itertools, pulp

# 1) Input rankings
user_order = [
    "Course-30","Course-12","Course-80","Course-22","Course-90",
    "Course-73","Course-70","Course-53","Course-4","Course-44",
    "Course-39","Course-45","Course-10","Course-76","Course-83",
    "Course-33","Course-77","Course-0","Course-31","Course-18"
]

platform_order = [
    "Course-30","Course-22","Course-33","Course-77","Course-31",
    "Course-4","Course-39","Course-18","Course-90","Course-45",
    "Course-83","Course-12","Course-53","Course-0","Course-70",
    "Course-76","Course-44","Course-73","Course-80","Course-10"
]

# Use user_order as the base course list
courses = user_order
assert set(user_order) == set(platform_order), "user_order and platform_order must match"

# 2) Rank look-ups
user_rank = {c: i for i, c in enumerate(user_order)}
platform_rank = {c: i for i, c in enumerate(platform_order)}

# 3) Weights
w_user, w_plat = 0.50, 0.50

# 4) Build pairwise preferences using directional voting and soft conflict penalties
m = {}
for i, j in itertools.combinations(courses, 2):
    sign_user = 1 if user_rank[i] < user_rank[j] else -1
    sign_plat = 1 if platform_rank[i] < platform_rank[j] else -1

    vote = w_user * sign_user + w_plat * sign_plat

    if vote > 0:
        m[(i, j)] = abs(vote)
    elif vote < 0:
        m[(j, i)] = abs(vote)
    else:
        # Conflict (user and platform disagree): add small penalty both ways
        m[(i, j)] = 0.1
        m[(j, i)] = 0.1

print("Total penalty edges:", len(m))
print("Sample margins:", list(m.items())[:5])

# 5) ILP Model
model = pulp.LpProblem("Kemeny", pulp.LpMinimize)
x = pulp.LpVariable.dicts('x', (courses, courses), 0, 1, cat='Binary')

# ✅ Penalize violations of directional preferences
model += pulp.lpSum(weight * x[j][i] for (i, j), weight in m.items())

# 6) Antisymmetry + totality
for i, j in itertools.permutations(courses, 2):
    model += x[i][j] + x[j][i] == 1

# 7) Transitivity
for i, j, k in itertools.permutations(courses, 3):
    model += x[i][j] + x[j][k] + x[k][i] >= 1

# 8) Solve
model.solve(pulp.PULP_CBC_CMD(msg=False))

# 9) Extract the consensus order
score = pulp.value(model.objective)
consensus_order = sorted(
    courses,
    key=lambda c: sum(x[c][d].value() for d in courses if d != c),
    reverse=True
)

# Output...show the results in the consensus order



Total penalty edges: 291
Sample margins: [(('Course-30', 'Course-12'), 1.0), (('Course-30', 'Course-80'), 1.0), (('Course-30', 'Course-22'), 1.0), (('Course-30', 'Course-90'), 1.0), (('Course-30', 'Course-73'), 1.0)]

Weighted-Kendall score = 10.10
Consensus ranking:
 1. Course-30    (user= 1, platform= 1)
 2. Course-22    (user= 4, platform= 2)
 3. Course-90    (user= 5, platform= 9)
 4. Course-12    (user= 2, platform=12)
 5. Course-80    (user= 3, platform=19)
 6. Course-73    (user= 6, platform=18)
 7. Course-70    (user= 7, platform=15)
 8. Course-53    (user= 8, platform=13)
 9. Course-4     (user= 9, platform= 6)
10. Course-39    (user=11, platform= 7)
11. Course-45    (user=12, platform=10)
12. Course-83    (user=15, platform=11)
13. Course-76    (user=14, platform=16)
14. Course-44    (user=10, platform=17)
15. Course-33    (user=16, platform= 3)
16. Course-77    (user=17, platform= 4)
17. Course-31    (user=19, platform= 5)
18. Course-18    (user=20, platform= 8)
19. Course-1